<a href="https://colab.research.google.com/github/vndee/pytorch-vi/blob/master/data_loading_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

#Data Loading and Processing Tutorial

**Author**: Sasank Chilamkurthy <https://chsasank.github.io>_

Một trong những vấn đề được đầu tư và bỏ công sức lớn trong việc giải quyết các bài toán học sâu đó là chuẩn bị dữ liệu. Do đó PyTorch cung cấp rất nhiều công cụ hỗ trợ cho việc load dữ liệu cũng xử lí dữ liệu một cách dễ dàng và trực quan. Trong bài này chúng ta sẽ tìm hiểu load và xử lý dữ liệu trong PyTorch thực sự hoạt động như thế nào.

Trước hết chúng ta cần cài một số package liên quan đến dữ liệu:
- `scikit-images`: Dùng để đọc và biến đổi ảnh.
- `pandas`:  Dùng để đọc file csv một cách dễ dàng.


In [0]:
import os
import torch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from __future__ import print_function, division

# ignore warning
import warnings
warnings.filterwarnings('ignore')

# iteractive mode
plt.ion()